# Tobaco comsumption per countries

### Hipotesis and exploration

Data source: IHME, Global Burden of Disease (2019)

OurWorldInData.org/smoking | CC BY

In [275]:
import pandas as pd
import requests
import getpass
import numpy as np
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [276]:
year = "2018"

In [277]:
df_18 = pd.read_csv(f"../data/SmokingAndTobaccoData{year}.csv")
df_18.sample()

,Location,CigaretteSmokingPrevalence,TobaccoSmokingPrevalence,TobaccoUsePrevalance,HealyWarningsOnSmoking,CheapestBrandCigaretteCurrency,MostSoldBrandCigaretteCurrency,MostSoldBrandCigarettePrice,PremiumBrandCigarettePrice,TreatmentInHealthFacilities,BanOnEducationalFacilites,BanOnGovernmentFacilites,BanOnHealthcareFacilites,BanOnIndoorOffices,BanOnPublicTransport,BanOnPubsAndBars,BanOnRestaurants,BanOnUniversities,RegulationsOnSmokeFreeEnvironments
135,Turkey,27.2 [21.7 – 33],29.3 [23.7 – 35.1],29.3 [23.7 – 35.1],No,7.5,TRY,13.5,13.5,Yes in some,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [278]:
df_18.dtypes

Location                               object
CigaretteSmokingPrevalence             object
TobaccoSmokingPrevalence               object
TobaccoUsePrevalance                   object
HealyWarningsOnSmoking                 object
CheapestBrandCigaretteCurrency         object
MostSoldBrandCigaretteCurrency         object
MostSoldBrandCigarettePrice            object
PremiumBrandCigarettePrice             object
TreatmentInHealthFacilities            object
BanOnEducationalFacilites             float64
BanOnGovernmentFacilites              float64
BanOnHealthcareFacilites              float64
BanOnIndoorOffices                    float64
BanOnPublicTransport                  float64
BanOnPubsAndBars                      float64
BanOnRestaurants                      float64
BanOnUniversities                     float64
RegulationsOnSmokeFreeEnvironments    float64
dtype: object

In [282]:
print(df_18["HealyWarningsOnSmoking"].unique())
print(df_18["TreatmentInHealthFacilities"].unique())

['No' 'Not applicable' 'Yes']
['Yes in some' 'No' 'Yes in most']


In [273]:
# drop non useful columns for the study
df_18 = df_18.drop(columns=["CheapestBrandCigaretteCurrency", "BanOnEducationalFacilites", "PremiumBrandCigarettePrice",	"BanOnGovernmentFacilites",	"BanOnHealthcareFacilites",	"BanOnIndoorOffices",	"BanOnPublicTransport",	"BanOnPubsAndBars",	"BanOnRestaurants",	"BanOnUniversities",	"RegulationsOnSmokeFreeEnvironments"])

In [ ]:
# Change column name
df_18 = df_18.rename(columns={'Location': 'Country'})

In [ ]:
# Make numeric
df_18["MostSoldBrandCigarettePrice"] = pd.to_numeric(df_18["MostSoldBrandCigarettePrice"], errors='coerce')

In [ ]:
df_18.dtypes

Country                            object
CigaretteSmokingPrevalence         object
TobaccoSmokingPrevalence           object
TobaccoUsePrevalance               object
HealyWarningsOnSmoking             object
MostSoldBrandCigaretteCurrency     object
MostSoldBrandCigarettePrice       float64
TreatmentInHealthFacilities        object
dtype: object

In [ ]:
api_key = getpass.getpass("Enter your token: ")

In [ ]:
def currency_conversion(currency, amount):
    base_url = f"https://api.currencyapi.com/v3/convert?apikey={api_key}"
    endpoint = f"&date={year}-06-01&base_currency={currency}&currencies=USD&value={amount}"
    url_for_request = base_url + endpoint
    res = requests.get(url_for_request)
    
    try:
        result = round(res.json()["data"]["USD"]["value"], 2)

        return result
    except :
        print(f"couldn't convert from {currency}, {url_for_request}")
        print(res.content)
        return np.nan

In [ ]:
df_18["MostSoldBrandCigarettePriceInUSD"] = df_18.apply(lambda x: currency_conversion(x["MostSoldBrandCigaretteCurrency"], x["MostSoldBrandCigarettePrice"]), axis=1)

couldn't convert from All, https://api.currencyapi.com/v3/convert?apikey=cur_live_AmzxX9c02B3yUdYzXJsBJgAQz5tBiGjcEQRIfTSm&date=2018-06-01&base_currency=All&currencies=USD&value=240.0
b'{"message":"Validation error","errors":{"base_currency":["The selected base currency is invalid."]},"info":"For more information, see documentation: https:\\/\\/currencyapi.com\\/docs\\/status-codes#_422"}'
couldn't convert from Data not available, https://api.currencyapi.com/v3/convert?apikey=cur_live_AmzxX9c02B3yUdYzXJsBJgAQz5tBiGjcEQRIfTSm&date=2018-06-01&base_currency=Data not available&currencies=USD&value=nan
b'{"message":"Validation error","errors":{"base_currency":["The selected base currency is invalid."],"value":["The value must be a number.","The value must be greater than 0."]},"info":"For more information, see documentation: https:\\/\\/currencyapi.com\\/docs\\/status-codes#_422"}'
couldn't convert from Not applicable, https://api.currencyapi.com/v3/convert?apikey=cur_live_AmzxX9c02B3yUdYzX

In [ ]:
df_18.sample()

,Country,CigaretteSmokingPrevalence,TobaccoSmokingPrevalence,TobaccoUsePrevalance,HealyWarningsOnSmoking,MostSoldBrandCigaretteCurrency,MostSoldBrandCigarettePrice,TreatmentInHealthFacilities,MostSoldBrandCigarettePriceInUSD
7,Azerbaijan,15.6 [9.4 – 21.3],19.6 [13.2 – 27.1],19.6 [13.2 – 27.1],Not applicable,AZN,1.2,No,0.71


In [ ]:
# Set up the WebDriver (for Chrome)
driver = webdriver.Chrome(executable_path='./chromedriver.exe')

# Navigate to the webpage
url = f"https://ourworldindata.org/grapher/number-of-deaths-by-risk-factor?tab=table&time={year}"
driver.get(url)

# Wait for the dynamic content to load (e.g., using an explicit wait)
wait = WebDriverWait(driver, 10)
element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'tbody')))

# Extract the table data
table_data = driver.find_element(By.CSS_SELECTOR, 'tbody').text

# Close the browser
driver.quit()

# Split the data into lines
lines = table_data.split('\n')

# Create a list of dictionaries to store the data
data_list = []

for line in lines:
    values = line.split()
    if len(values) >= 17:  # Check if there are at least 17 values in the list
        country_parts = []
        numbers = []
        for value in values:
            if value.isalpha():
                country_parts.append(value)
            else:
                numbers.append(value)
        country = ' '.join(country_parts)  # Join the word parts to form the country name
        deaths_by_smoking = numbers[13]  # Assuming you want the 13th value (0-based index) from the numbers
        data_list.append({'Country': country, 'DeathsBySmoking': deaths_by_smoking})

# Create a DataFrame from the list of dictionaries
sub_df_deaths = pd.DataFrame(data_list)


In [ ]:
# Set up the WebDriver (for Chrome)
driver = webdriver.Chrome(executable_path='./chromedriver.exe')

# Navigate to the webpage
url = f"https://ourworldindata.org/grapher/gdp-per-capita-maddison?tab=table&time={year}"
driver.get(url)

# Wait for the dynamic content to load (e.g., using an explicit wait)
wait = WebDriverWait(driver, 10)
element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'tbody')))

# Extract the table data
table_data = driver.find_element(By.CSS_SELECTOR, 'tbody').text

# Close the browser
driver.quit()

# Split the data into lines
lines = table_data.split('\n')

# Initialize lists for countries and GDP values
countries = []
gdp_values = []

# Regular expression pattern to match GDP values with dollar signs and commas
gdp_pattern = r'\$([\d,]+)'

# Iterate through the lines and extract country names and GDP values. Last 21 values not valuable data because they refer to continents and general GDP
for i in range(0, len(lines) -21, 2):
    country = lines[i]
    gdp_match = re.search(gdp_pattern, lines[i + 1])
    
    if gdp_match:
        gdp = int(gdp_match.group(1).replace(',', ''))
    else:
        gdp = None
    
    countries.append(country)
    gdp_values.append(gdp)

# Create a DataFrame
sub_df_gdp = pd.DataFrame({'Country': countries, 'GDPinUSD': gdp_values})

In [ ]:
# Set up the WebDriver (for Chrome)
driver = webdriver.Chrome(executable_path='./chromedriver.exe')

# Navigate to the webpage
url = f"https://www.populationpyramid.net/population-size-per-country/{year}/"
driver.get(url)

# Wait for the dynamic content to load (e.g., using an explicit wait)
wait = WebDriverWait(driver, 10)
element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'tbody')))

# Extract the table data
table_data = driver.find_element(By.CSS_SELECTOR, 'tbody').text

# Close the browser
driver.quit()

# Define a regular expression pattern to capture the rank, country name, and population
pattern = re.compile(r'(\d+)\n(.+?)\s([0-9,]+)')

matches = re.findall(pattern, table_data)

data_list = []

for match in matches:
    rank = match[0]
    country = match[1]
    population = match[2].replace(',', '')  # Remove commas from the population number
    data_list.append({'Rank': rank, 'Country': country, 'Population': population})

# Create a DataFrame from the list of dictionaries
sub_df_population = pd.DataFrame(data_list)

#drop rank column
sub_df_population.drop(columns=['Rank'], inplace=True)


In [ ]:
sub_df_deaths.sample()

,Country,DeathsBySmoking
0,Afghanistan,"9,975"


In [ ]:
sub_df_gdp.sample()

,Country,GDPinUSD
59,Greece,23451


In [ ]:
sub_df_population.sample()

,Country,Population
180,French Polynesia,297606


In [ ]:
# Merge new information
df_18 = df_18.merge(sub_df_deaths, on='Country', how='left')
df_18 = df_18.merge(sub_df_gdp, on='Country', how='left')
df_18 = df_18.merge(sub_df_population, on='Country', how='left')

In [ ]:
df_18.sample()


,Country,CigaretteSmokingPrevalence,TobaccoSmokingPrevalence,TobaccoUsePrevalance,HealyWarningsOnSmoking,MostSoldBrandCigaretteCurrency,MostSoldBrandCigarettePrice,TreatmentInHealthFacilities,MostSoldBrandCigarettePriceInUSD,DeathsBySmoking,GDPinUSD,Population
80,Madagascar,14.7 [10.4 – 19.9],20.6 [12.5 – 29.1],28.9 [20.5 – 37.9],No,MGA,NaN,Yes in some,NaN,"5,312",1428.0,26846541


In [ ]:
df_18.dtypes

Country                              object
CigaretteSmokingPrevalence           object
TobaccoSmokingPrevalence             object
TobaccoUsePrevalance                 object
HealyWarningsOnSmoking               object
MostSoldBrandCigaretteCurrency       object
MostSoldBrandCigarettePrice         float64
TreatmentInHealthFacilities          object
MostSoldBrandCigarettePriceInUSD    float64
DeathsBySmoking                      object
GDPinUSD                            float64
Population                           object
dtype: object

In [ ]:
df_18.isnull().sum()

Country                              0
CigaretteSmokingPrevalence           0
TobaccoSmokingPrevalence             0
TobaccoUsePrevalance                 0
HealyWarningsOnSmoking               0
MostSoldBrandCigaretteCurrency       0
MostSoldBrandCigarettePrice         26
TreatmentInHealthFacilities          0
MostSoldBrandCigarettePriceInUSD    28
DeathsBySmoking                     12
GDPinUSD                            26
Population                          13
dtype: int64